In [107]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots
using HDF5

N = 24

f4 = h5open("Basis states Final/MPO_$N.h5","r")
W1 = read(f4,"W",MPO)
close(f4)

# Assuming `siteinds(W1)` gives the indices of the MPO
all_sites = siteinds(W1)

# Consider only physical sites
s = [pair[2] for pair in all_sites]

# println(s)

24-element Vector{Index{Int64}}:
 (dim=2|id=720|"Qubit,Site,n=1")
 (dim=2|id=282|"Qubit,Site,n=2")
 (dim=2|id=865|"Qubit,Site,n=3")
 (dim=2|id=206|"Qubit,Site,n=4")
 (dim=2|id=772|"Qubit,Site,n=5")
 (dim=2|id=971|"Qubit,Site,n=6")
 (dim=2|id=463|"Qubit,Site,n=7")
 (dim=2|id=326|"Qubit,Site,n=8")
 (dim=2|id=267|"Qubit,Site,n=9")
 (dim=2|id=136|"Qubit,Site,n=10")
 (dim=2|id=920|"Qubit,Site,n=11")
 (dim=2|id=185|"Qubit,Site,n=12")
 (dim=2|id=329|"Qubit,Site,n=13")
 (dim=2|id=712|"Qubit,Site,n=14")
 (dim=2|id=310|"Qubit,Site,n=15")
 (dim=2|id=707|"Qubit,Site,n=16")
 (dim=2|id=936|"Qubit,Site,n=17")
 (dim=2|id=775|"Qubit,Site,n=18")
 (dim=2|id=313|"Qubit,Site,n=19")
 (dim=2|id=875|"Qubit,Site,n=20")
 (dim=2|id=330|"Qubit,Site,n=21")
 (dim=2|id=354|"Qubit,Site,n=22")
 (dim=2|id=782|"Qubit,Site,n=23")
 (dim=2|id=725|"Qubit,Site,n=24")

In [108]:
# Taking the input 

x = range(0, stop=2π, length=2^(N))
input_array = cos.(x)


array = input_array / norm(input_array) # Input
ITensors.disable_warn_order()

cutoff1 = 1E-24
maxdim1 = 10


T = ITensor(array,s)

ψ = MPS(T,s;cutoff=cutoff1,maxdim=maxdim1)

orthogonalize!(ψ, 1) # Orthogonalize Psi 

start = time()

result = contract(W1, ψ)
end_time = time()
time1 = end_time - start

println(time1)

0.0064849853515625


In [109]:
using FFTW

start2 = time()
x2 = fft(array)
end2 = time()

time2 = end2 - start2
println(time2)

0.3464231491088867


# Check Output 

In [110]:

inner_products = []
MPS2 = []

for i in 1:2^3
    f2 = h5open("Basis states Final/MPS_$N/MPS_create_$i.h5","r")
    mps1 = read(f2,"M",MPS)
    push!(MPS2,mps1)
    close(f2)
end

for i in 1:(2^3)
    push!(inner_products, inner(MPS2[i], result))
    print(inner(MPS2[i], result)*2^(N/2))
    println()
end


0.00011508898252770066 - 1.4131360928077707e-11im
2896.309375740525 + 0.0005423417057245976im
-1.7149549837180201e-12 - 1.7300819529033038e-18im
-4.31582597154179e-5 - 5.691968938550152e-11im
-5.716498401355954e-13 + 6.929061661496108e-19im
-1.4386040742234846e-5 + 1.2977321627960134e-11im
-2.8582174688274587e-13 + 6.215067512411919e-18im
-7.192940515338487e-6 + 1.5417721857515038e-10im


In [111]:
for i in 1:2^3
    println(x2[i])
end

0.00034526697248534784 + 0.0im
2896.3093757400056 + 0.0005423441083053668im
-0.00011508900912163473 - 4.3101592978565534e-11im
-4.3158377390191806e-5 - 2.424130757637164e-11im
-2.3017801092445136e-5 - 1.724064385059472e-11im
-1.4386125709308547e-5 - 1.3462128389616158e-11im
-9.864771851831875e-6 - 1.1083323426692858e-11im
-7.193062755537335e-6 - 9.437020888510453e-12im


# Code to generate basis states to measure

In [112]:
# function generate_basis_states(n)
#     basis_states = []
#     for i in 0:(2^n - 1)
#         binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
#         push!(basis_states, collect(binary_str))  # Collect splits string into array of characters
#     end
#     return basis_states
# end

# MPS_states = []
# basis_states = generate_basis_states(N)

# for i in 1:(2^N)
#     push!(MPS_states, MPS(s,string.(basis_states[i])))
# end

# using HDF5
# f = 0 

# for (i, mps) in enumerate(MPS_states)
#     f = h5open("Basis states Final/MPS_$N/MPS_create_$i.h5","w")
#     write(f,"M", mps)
# end
# close(f)